![alt text](STAR_MELT_logo.png "STAR_MELT")

# STAR-MELT Example Notebook

Welcome to the example notebook for STAR-MELT.\
This notebook will use the example FITS files to show you how to use STAR-MELT on your own data.\
Additional functions at the end of the notebook can be copied to where needed.

### Notebook tips:
* ***Shift + Enter on a code cell/block to run it and advance to the next cell.***
* Can re-run blocks out of order as long as the variables are already there
* To add text notes between blocks, click on the [number] left of a block to select the block (not the code within the block) and press 'b' to make a new cell below, then with the new cell highlighted press 'm' to switch it to markdown instead of code, type your notes then 'shift+enter' 'runs' it to generate the text


In [ ]:
#Packages required by STAR-MELT, some are not used directly in this notebook, but called by the modules, check that they are all installed here
import time
from matplotlib import *
from matplotlib.pyplot import *
import numpy as np
import pandas as pd
import astropy
from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord, EarthLocation
from astropy.stats import sigma_clip
from astroquery.simbad import Simbad
from astropy.timeseries import LombScargle
import numpy.ma as ma
import os
from PyAstronomy import pyasl
from lmfit.models import GaussianModel, LinearModel
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
from scipy.signal import find_peaks_cwt
from scipy.signal import argrelextrema
from ESO_fits_get_spectra import *
from ESP_fits_get_spectra import *
from utils_data import *
from utils_spec import *
from utils_physics import *
import utils_shared_variables as USH
from utils_saha_av import *
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widget
from IPython.display import display,clear_output
import qgrid

#### If you change which %matplotlib magic command here you must restart the notebook

In [ ]:
#for notebook/slides
%matplotlib notebook 
#for lab
#%matplotlib widget
#for new window plots
#%matplotlib qt
%reload_ext autoreload
%autoreload 2

In [ ]:
rcParams.update({'figure.max_open_warning': 50})
rcParams['figure.dpi'] = 100
matplotlib.rc('font', family='serif',size=14)
USH.fig_size_s=(8,6)
USH.fig_size_l=(9,5)
USH.fig_size_n=(9,3)

In [ ]:
# The line table used in the emission line matching will be automatically selected 
# depending on which instrument is selected. This block is just to view the line list.
line_table=USH.line_table
line_table=USH.line_table_prev_obs

#uncomment below if you want to see the line table in qgrid
#qgrid_widget1 = qgrid.show_grid(line_table, show_toolbar=False,grid_options={'forceFitColumns': False, 'defaultColumnWidth': 75})
#display(qgrid_widget1)

### Select star from available data

* Data should be organised as /STAR/INSTRUMENT/spectrum.fits or STAR/spectrum.fits
* Can run organise_fits_files(dir_of_files,output_dir) on dir_of_files containing .fits files that can be read by the read in scripts (example block near end of notebook)

In [ ]:
standards_dir='Standard_stars' 
data_dir='Example_data' #you can change this path to your local path of data

star_list=os.listdir(data_dir)
try:
    star_list.remove('.DS_Store')#remove temp file on mac because python will think it's a star!
except:
    pass

star_select = widget.Dropdown(
    options=sorted(star_list),
    #value='GQ_Lup',
    description='Select star:',
)
display(star_select)


### Find closest standard star by checking spectral types on SIMBAD, get radial velocity of standard star

* Look up star in SIMBAD and get spectral type 
* Match to closest spectral type template star from available data in standards_dir
* Retrieve radial velocity and Vsini of template star
* If simbad_out=True, also return simbad query table

In [ ]:
target=star_select.value
data_fits_files=get_files(os.path.join(data_dir,target),'.fits','.FTZ')
standard_fits_files,mk,stype,st_rv,simbad=get_fits_files_simbad(target,standards_dir,simbad_out=True)

### Read in data and see what data is available for target star

In [ ]:
data_dates_range1,instrument,w0=get_instrument_date_details(data_fits_files,qgrid=True)
qgrid_widget1 = qgrid.show_grid(data_dates_range1, show_toolbar=False)
display(qgrid_widget1)

### Selecting the instrument instructions:
* The qgrid above will display all data read in for the selected target
* You can use the columns in qgrid to make selections or filter ranges
* Use the button options below to decide how to make the selections
* Select 'qgrid filtered' to use all data shown in the above gqrid after using the column buttons to filter, e.g by inst and/or wmin/wmax
* Select 'qgrid selected' to use all data highlighted in the above gqrid (shift/ctrl/cmd + click to highlight)
* Or select individual instrument to use all data from that instrument only


In [ ]:
inst_options=['qgrid filtered','qgrid selected']
inst_options.extend(unique(data_dates_range1.inst).tolist())
inst_select = widget.RadioButtons(
    options=inst_options,
    description='Instrument selection:',
    style = {'description_width': 'initial'}
)
display(inst_select)

### Average data frame
* If one instrument selected, this will auto run the average dataframe
* If one instrument with >1 spectral arm selected, the arm must be specified when prompted
* If >1 inst selected, refine wavelength coverage above so it's common between instruments, then specify this coverage when prompted


In [ ]:
instr_select='any' 
if inst_select.value=='qgrid filtered':
    new_list=qgrid_widget1.get_changed_df()
elif inst_select.value=='qgrid selected':
    new_list=qgrid_widget1.get_selected_df()
else:
    instr_select=inst_select.value
    new_list=data_dates_range1[data_dates_range1.inst==instr_select]
    
all_inst=True if len(unique(new_list.inst)) >1 else False

data_dates_range,instrument,w0=get_instrument_date_details(new_list.file,instr_select,
                                                           all_inst=all_inst,qgrid=False)#,start_date='2008',end_date='2012')
#w0=np.arange(min(w0),max(w0),1)
df_av=get_av_spec(data_dates_range,w0,norm=False,output=True,plot_av=True,savefig=False)#,label='utc_inst')
df_av_norm=get_av_spec(data_dates_range,w0,norm=True,output=True,plot_av=True)#,label='utc_inst')

USH.target=target
USH.instrument=unique(new_list['inst'])

#### *Only run the following block to specify a further wavelength exclusion, otherwise can be skipped*

In [ ]:
w0,df_av,df_av_norm=wl_excluder(w0,df_av,df_av_norm,w0_cut=[[3000,3860],[6800,6970], [8150,8390], [9100,9250]])
wl_plot(df_av_norm,plot_av=False)

### Saved emission lines load in

* This should be a list of lines identified with STAR-MELT for the star of interest

In [ ]:
em_file_r='Line_Resources/EX_Lupi_FEROSHARPS_2sig_poly3_win2_april16_lines_revised.csv' #example provided for EX Lupi (JCW+2021)
em_loaded = pd.read_csv(em_file_r,delimiter=',')#.astype('object')

qgrid_widget_em = qgrid.show_grid(em_loaded, show_toolbar=False,grid_options={'forceFitColumns': False, 'defaultColumnWidth': 75})
qgrid_widget_em

#### If the loaded in list is refined in the qgrid, update em_loaded with below block

In [ ]:
#em_loaded=qgrid_widget_em.get_selected_df()
em_loaded=qgrid_widget_em.get_changed_df()

#### Plot the loaded in list of lines with the average spectra

In [ ]:
plot_em_lines(df_av_norm,em_loaded,plot_av=False,fs=USH.fig_size_n)

## Calculate the radial velocity and Vsini of the target star
### Please note that RV and Vsini calculations are currently not working as intended for data that uses vacuum wavelengths.

* *Get the radial velocity and vsini of the target star using template star and CCF*
* Best not to use normalised average flux data frame, default is to use standard average flux
* May need to change w_min and w_max
* Recommended to use high resolution spectra for CCF with HARPS/FEROS templates

In [ ]:
radvel=32 #you can set the values manually here and comment out the radvel,vsini= line
vsini=200
st_info,st_wave,st_flux,st_err=read_fits_files(standard_fits_files[0],verbose=True) 
radvel,vsini=get_rv_vsini(df_av,st_wave,st_flux,st_rv,date='med_flux',w_min=5000,w_max=5500)
USH.radvel=radvel #do not comment these out, other functions need the values
USH.vsini=vsini

## Continuum subtraction
* *Get normalised average flux data frame and subtract continuum, plot on screen*
* Use the poly order slider to adjust the degree of the polynomial for the SG filter
* Use the wl_win slider to adjust the wavelength window of the SG filter
* For data with more noise, use a lower order and/or larger wl_win
* Flux and Wave sliders are just for the plot limits

In [ ]:
def view_cont(poly=1, wl_win=1,plot_y=[],plot_x=[]):
    global f_flat
    f_flat=subtract_cont(df_av_norm,av='med',poly=poly,wl_win=wl_win,output=True,plot_x=plot_x,plot_y=plot_y)


interact(view_cont, 
         poly=widget.IntSlider(min=1, max=10,step=1,value=1,layout={'width': '600px'},continuous_update=False), 
         wl_win=widget.FloatSlider(min=0.5, max=20, step=0.5,value=1,layout={'width': '600px'},continuous_update=False), 
         plot_y=widget.FloatRangeSlider(min=0,max=max(df_av_norm.med_flux),
            step=1,value=[0,max(df_av_norm.med_flux)],
            layout={'width': '600px'},description='Flux',continuous_update=False),
         plot_x=widget.FloatRangeSlider(min=min(df_av_norm.wave),max=max(df_av_norm.wave),
            step=10,value=[min(df_av_norm.wave),max(df_av_norm.wave)],
            layout={'width': '800px'},description='Wave [Angstroms]',
            style = {'description_width': 'initial'},continuous_update=False)
        )


## Automatic emission line identification
* *Find the emission lines*
* Use the sigma slider to select the threshold at which to match the identified lines
* Wave slider here does limit the output for which lines are matched, only between wave limits

In [ ]:
def get_lines(sigma=3,xrange=[]):
    global em_matches,em_match_common_Ek
    em_matches,em_match_common_Ek=find_em_lines(df_av,f_flat,radvel,vsini=vsini,sigma=sigma,
                                                av='med',atol=0.5,output=True,line_id=True,xrange=xrange)
    print('number of emission lines matched:',len(em_matches)) 
    
interact(get_lines, 
         sigma=widget.FloatSlider(min=0.5, max=15, step=0.5,value=3,layout={'width': '600px'},continuous_update=False), 
         xrange=widget.FloatRangeSlider(min=min(df_av_norm.wave),max=max(df_av_norm.wave),
            step=5,value=[min(df_av_norm.wave),max(df_av_norm.wave)],
            layout={'width': '800px'},description='Wave [Angstroms]',
            style = {'description_width': 'initial'},continuous_update=False)
        )   



# The following sections are for fitting and analysis
# There are sections for periodogram/velocity analsysis and for the Saha/Sobolev analysis

### Select here whether to use the loaded in emission lines, or the emission lines identified within run, or the lines identfied after fitting from line_results

In [ ]:
em_matches_selection=widget.RadioButtons(
    options=['Loaded in list of lines', 'Lines identified here','Lines selected after fitting'],
    value='Lines identified here',
    layout={'width': 'max-content'}, 
    style = {'description_width': 'initial'},
    description='Select em list:',
)
display(em_matches_selection)

# Line selection and fitting
* Either filter or select from the qgrid, and pick corresponded option in block below with the fitting options


In [ ]:
if em_matches_selection.value=='Loaded in list of lines':
    df = em_loaded
elif em_matches_selection.value=='Lines selected after fitting':
    df=em_loaded[em_loaded.obs_wl_air.isin(line_results.obs_wl_air)]
else:
    df = em_matches
    
qgrid_widget = qgrid.show_grid(df, show_toolbar=False,grid_options={'forceFitColumns': False, 'defaultColumnWidth': 75})
display(qgrid_widget)

In [ ]:
#df.to_csv(target+'_3sig_1poly_5wl.csv',index=False) #this is how you can save the output of the lines, which can then be read back in as em_loaded earlier on


### Line fitting options
* Use the select dates widget to pick all observation dates you want for the periodogram, drag click or shift click
* Options for number of Gaussians, 1, 2 or 3.
* If neg=True, one positive, one negative Gauss fitted, if neg=True and ngauss=2, two positive, one negative Gauss fitted etc
* May need to increase gof_min for >1 Gauss, defualt is gof_min=0.2
* For periodogram analysis, do not use av_flux or med_flux
* For Saha use only Fe or He, for Sobolov use only Fe (to be updated with more species)


In [ ]:
line_select = widget.RadioButtons(options=['qgrid filtered','qgrid selected'],description='Line selection:',style = {'description_width': 'initial'})

date_selector = widget.SelectMultiple(options=df_av.columns[1:-1],value=['med_flux'],description='Select dates:',)
display(widget.HBox([line_select,date_selector]))

vel_range=widget.FloatText(value='50',step=10,description='Velocity window',layout={'width': 'max-content'}, style = {'description_width': 'initial'})


ngauss_sel=widget.IntSlider(value='1',min=1,max=3,description='# of positive Gauss:', style = {'description_width': 'initial'})
neg_sel=widget.Checkbox(value=False,description='Include a negative Gauss?',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
vred_sel=widget.Checkbox(value=False,description='Include a Vred calc.?',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
gof_min_sel=widget.FloatSlider(min=0.1, max=1, step=0.1,value=0.2,description='GoF min:', style = {'description_width': 'initial'})
reject_low_gof=widget.Checkbox(value=True,description='Use min GoF?',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
reject_line_close=widget.Checkbox(value=True,description='Use min std err?',layout={'width': 'max-content'}, style = {'description_width': 'initial'})

set_g1_cen=widget.Checkbox(value=False,description='Set G1 centre limits?',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
set_g1_sig=widget.Checkbox(value=False,description='Set G1 sigma limits?',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
g1_cen_min=widget.FloatText(value='0',step=10,description='G1 centre min',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
g1_cen_max=widget.FloatText(value='20',step=10,description='max',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
g1_sig_min=widget.FloatText(value='0',step=10,description='G1 sigma min',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
g1_sig_max=widget.FloatText(value='50',step=10,description='max',layout={'width': 'max-content'}, style = {'description_width': 'initial'})

set_g2_cen=widget.Checkbox(value=False,description='Set G2 centre limits?',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
set_g2_sig=widget.Checkbox(value=False,description='Set G2 sigma limits?',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
g2_cen_min=widget.FloatText(value='10',step=10,description='G2 centre min',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
g2_cen_max=widget.FloatText(value='50',step=10,description='max',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
g2_sig_min=widget.FloatText(value='0',step=10,description='G2 sigma min',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
g2_sig_max=widget.FloatText(value='100',step=10,description='max',layout={'width': 'max-content'}, style = {'description_width': 'initial'})

set_g3_cen=widget.Checkbox(value=False,description='Set G3 centre limits?',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
set_g3_sig=widget.Checkbox(value=False,description='Set G3 sigma limits?',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
g3_cen_min=widget.FloatText(value='10',step=10,description='G3 centre min',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
g3_cen_max=widget.FloatText(value='50',step=10,description='max',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
g3_sig_min=widget.FloatText(value='0',step=10,description='G3 sigma min',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
g3_sig_max=widget.FloatText(value='100',step=10,description='max',layout={'width': 'max-content'}, style = {'description_width': 'initial'})

set_g4_cen=widget.Checkbox(value=False,description='Set neg Gauss centre limits?',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
set_g4_sig=widget.Checkbox(value=False,description='Set neg Gauss sigma limits?',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
g4_cen_min=widget.FloatText(value='10',step=10,description='neg centre min',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
g4_cen_max=widget.FloatText(value='50',step=10,description='max',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
g4_sig_min=widget.FloatText(value='0',step=10,description='neg sigma min',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
g4_sig_max=widget.FloatText(value='100',step=10,description='max',layout={'width': 'max-content'}, style = {'description_width': 'initial'})

savefig=widget.Checkbox(value=False,description='Save all output plots to file?',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
title=widget.Dropdown(options=['full','simple','none'],value='full',description='Plot title',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
df_sel=widget.Dropdown(options=['df_av','df_av_norm'],value='df_av',description='Use df_av or df_av_norm?',layout={'width': 'max-content'}, style = {'description_width': 'initial'})

display(df_sel,vel_range)
display(widget.TwoByTwoLayout(top_left=widget.HBox([reject_low_gof,reject_line_close]),top_right=gof_min_sel,bottom_left=ngauss_sel,bottom_right=widget.HBox([neg_sel,vred_sel])))
display(widget.TwoByTwoLayout(bottom_left=widget.HBox([g1_cen_min,g1_cen_max]),bottom_right=widget.HBox([g1_sig_min,g1_sig_max]),top_left=set_g1_cen,top_right=set_g1_sig))
display(widget.TwoByTwoLayout(bottom_left=widget.HBox([g2_cen_min,g2_cen_max]),bottom_right=widget.HBox([g2_sig_min,g2_sig_max]),top_left=set_g2_cen,top_right=set_g2_sig))
display(widget.TwoByTwoLayout(bottom_left=widget.HBox([g3_cen_min,g3_cen_max]),bottom_right=widget.HBox([g3_sig_min,g3_sig_max]),top_left=set_g3_cen,top_right=set_g3_sig))
display(widget.TwoByTwoLayout(bottom_left=widget.HBox([g4_cen_min,g4_cen_max]),bottom_right=widget.HBox([g4_sig_min,g4_sig_max]),top_left=set_g4_cen,top_right=set_g4_sig))
display(widget.HBox([title,savefig]))
    

* *The next block runs the fitting for the lines and options selected above*
* *The option to output each plot on screen is given*


## Fit the lines

In [ ]:
line_date_list=date_selector.value
if line_select.value=='qgrid filtered':
    em_lines=qgrid_widget.get_changed_df()
elif line_select.value=='qgrid selected':
    em_lines=qgrid_widget.get_selected_df()
print('number of lines attemping to fit %s:'%(len(em_lines)))
print('number of dates attempting to fit %s'%(len(line_date_list)))

#settings for Vred H balmer: g1 cen -75,50, neg cen 150,250, neg sig 0,250
#settings for Vred Ca II k: g1 cen -20,20, g2 cen -30,20 neg cen 150,250, neg sig 0,250

g1_cen=[g1_cen_min.value,g1_cen_max.value] if set_g1_cen.value==True else None #0,150 for CaK + H
g2_cen=[g2_cen_min.value,g2_cen_max.value] if set_g2_cen.value==True else None #-20,100 for CaK
g3_cen=[g3_cen_min.value,g3_cen_max.value] if set_g3_cen.value==True else None #-20,100 for CaK
neg_cen=[g4_cen_min.value,g4_cen_max.value] if set_g4_cen.value==True else None #0,50 for He, 200,50 for CaK + H
g1_sig=[g1_sig_min.value,g1_sig_max.value] if set_g1_sig.value==True else None #0,50 for He, 200,50 for CaK + H
g2_sig=[g2_sig_min.value,g2_sig_max.value] if set_g2_sig.value==True else None #0,50 for He, 200,50 for CaK + H
g3_sig=[g3_sig_min.value,g3_sig_max.value] if set_g3_sig.value==True else None #0,50 for He, 200,50 for CaK + H
neg_sig=[g4_sig_min.value,g4_sig_max.value] if set_g4_sig.value==True else None #0,50 for He, 200,50 for CaK + H

df_fit=df_av if df_sel.value=='df_av' else df_av_norm
    
em_line_date_results,em_line_date_results_common_Ek=get_line_results(em_lines,df_fit,line_date_list,target,radvel=radvel,
                                                                     w_range=vel_range.value,ngauss=ngauss_sel.value,output=show_output('individual em lines plots?'),
                                                                     g1_cen=g1_cen,g2_cen=g2_cen,g3_cen=g3_cen,neg_cen=neg_cen,g1_sig=g1_sig,g2_sig=g2_sig,g3_sig=g3_sig,neg_sig=neg_sig,
                                                                     neg=neg_sel.value,gof_min=gof_min_sel.value,title=title.value,vred=vred_sel.value,plot_comps=True,
                                                                     savefig=savefig.value,reject_low_gof=reject_low_gof.value,reject_line_close=reject_line_close.value)


In [ ]:
#em_line_date_results.to_csv(target+'_3sig_1poly_5wl_line_fits.csv',index=False) #this will save the fit results to file

# Periodogram/Variability analysis

## Identified line properties & fit parameters

In [ ]:
qgrid_widget3 = qgrid.show_grid(em_line_date_results, show_toolbar=False,grid_options={'forceFitColumns': False, 'defaultColumnWidth': 75})
qgrid_widget3

* If your data includes any observations from FEROS, an updated barycentric correction can be applied by running the block below, otherwise skip this block

In [ ]:
em_line_date_results=apply_bary_cor_FEROS(data_dates_range,simbad,em_line_date_results)
qgrid_widget3 = qgrid.show_grid(em_line_date_results, show_toolbar=False,grid_options={'forceFitColumns': False, 'defaultColumnWidth': 75})
qgrid_widget3

## Select options for phase plot and periodogram for a given fitted line

In [ ]:
line_selector = widget.Select(options=unique(em_line_date_results.obs_wl_air),description='Select line:',)
period_sel=widget.FloatText(value='7',step=0.5,description='Period to wrap:',layout={'width': 'max-content'}, style = {'description_width': 'initial'})

mjd0_sel=widget.Dropdown(options=unique(em_line_date_results.mjd),description='Sel mjd0:',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
mjmin_sel=widget.Dropdown(options=unique(em_line_date_results.mjd),description='Sel min mjd:',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
mjmax_sel=widget.Dropdown(options=unique(em_line_date_results.mjd),value=max(unique(em_line_date_results.mjd)),description='Sel max mjd:',layout={'width': 'max-content'}, style = {'description_width': 'initial'})

gofmin_sel=widget.FloatText(value='0.2',step=0.05,description='Min GoF:',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
errmin_sel=widget.FloatText(value='5',step=0.5,description='Min Std_err:',layout={'width': 'max-content'}, style = {'description_width': 'initial'})

filmin_sel=widget.FloatText(value='-20',step=1,description='Min Vel.:',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
filmax_sel=widget.FloatText(value='20',step=1,description='Max Vel.:',layout={'width': 'max-content'}, style = {'description_width': 'initial'})

maxper_sel=widget.FloatText(value='20',step=1,description='Max Period.:',layout={'width': 'max-content'}, style = {'description_width': 'initial'})
minper_sel=widget.FloatText(value='1',step=1,description='Min Period.:',layout={'width': 'max-content'}, style = {'description_width': 'initial'})


display(widget.HBox([line_selector,period_sel]))
display(widget.HBox([mjd0_sel,mjmin_sel,mjmax_sel]))
display(widget.HBox([gofmin_sel,errmin_sel]))
display(widget.HBox([filmin_sel,filmax_sel]))
display(widget.HBox([minper_sel,maxper_sel]))


#### Plot the phase and periodogram for given selections

In [ ]:
phase_period(em_line_date_results,linewav=line_selector.value,mjd0=mjd0_sel.value,period=period_sel.value,
             gofmin=gofmin_sel.value,filmin=filmin_sel.value,filmax=filmax_sel.value,maxper=maxper_sel.value,
             minper=minper_sel.value,errmin=errmin_sel.value,mjmin=mjmin_sel.value,mjmax=mjmax_sel.value)

# Saha & Sobolev analysis
* Run / re-run the line fitting for Fe or He lines for Saha, Fe lines for Sobolev (more species to be included)
* Best to use med_flux or av_flux for line fittings to work out ratios with higher s/n


## Identified line properties & fit parameters

In [ ]:
#em_line_date_results.drop_duplicates(subset='w0',inplace=True)
qgrid_widget_ss2 = qgrid.show_grid(em_line_date_results, show_toolbar=False,grid_options={'forceFitColumns': False, 'defaultColumnWidth': 75})
qgrid_widget_ss2

# Saha

## Highlight the lines from the above qgrid to send to the Saha function
* Must be Fe or He for Saha (more to be added)


In [ ]:
line_results=qgrid_widget_ss2.get_selected_df()
line_results

In [ ]:
saha_av(line_results,N_range=[5,22],T_range=[1000,2e4]) #max T is ~6.9e4

# Sobolev
* Only Fe lines should be fitted, more to be added
* Sobolev LGV approx requires lines from same upper energy level

## The qgrid below shows the lines from the same upper level, note that only one date/average should be shown in mjd

In [ ]:
sob_em_line_date_results_common_Ek=em_line_date_results_common_Ek[em_line_date_results_common_Ek.element=='Fe']
qgrid_widget_sob = qgrid.show_grid(sob_em_line_date_results_common_Ek, show_toolbar=False,grid_options={'forceFitColumns': False, 'defaultColumnWidth': 75})
qgrid_widget_sob

In [ ]:
line_results=qgrid_widget_sob.get_changed_df()
#line_results=qgrid_widget_sob.get_selected_df()

* *Run the sobolev function for the applicable lines*

In [ ]:
#use qgruid to update above df by either Fe I or Fe II, still need U(T) for other elements
sobolev_by_element(line_results,target,N_range=[6,22],T_range=[4e3,4e4],output=True,savefig=False)
    

# Extra functions from STAR MELT
* can copy and paste these blocks to use earlier on if needed, e.g. fitting for individual lines

### Organise fits files by target name
* This function will also check whether your fits files are compatible with the STAR-MELT read in scripts

In [ ]:
#This function will organise a directory of fits files into star/instr/obs.fits
datadir2='/Users/Documents/fits'
outputdir='/Users/Documents/HST_data'
organise_fits_files(datadir2,outputdir)

### View/fit individual lines from the loaded in list of lines and the df_av dataframe

In [ ]:
line_select1 = widget.Dropdown(
    options=em_loaded.rv_wl,
    description='Select line:',
)
display(line_select1)

In [ ]:
line=line_select1.value
vel_range=450
w_range=4
df_av_line_vel=get_line_spec(df_av,line,vel_range,vel_offset=0,vel=True)
df_av_line=get_line_spec(df_av_norm,line,w_range,vel_offset=0,vel=False)
vel_plot(df_av_line_vel,line=line,plot_av=False,plot_sd=False,fs=USH.fig_size_s)
#wl_plot(df_av_line,plot_av=True,fs=USH.fig_size_s)

In [ ]:
date_selector1 = widget.SelectMultiple(
    options=df_av_line_vel.columns[1:-1],
    value=['med_flux'],
    description='Select dates:',
)
display(date_selector1)

In [ ]:
line_date_list=date_selector1.value
g1_cen=None#[0,30]
g2_cen=None#[200,100]
neg_cen=None#[0,50]
g1_sig=None
g2_sig=None
neg_sig=None#[100,200]
out,x,y,line_info=gauss_stats(df_av_line_vel,line_date_list[0],ngauss=2,neg=False,vred=False,
                              g1_cen=g1_cen,g2_cen=g2_cen,neg_cen=neg_cen,g1_sig=g1_sig,g2_sig=g2_sig,neg_sig=neg_sig,
                              output=True,printout=False,subplot=False,title=None)